# IGC Parser Cheat Sheet (Inverse Gas Chromatography)

## 0) Entry point: parse the file
Use this in a notebook or script to load the CSV and split it into tables.

```python
from Parser import parse_sugar_se
result = parse_sugar_se()  # reads Data/Sugar SE.csv by default
```

---

## 1) What `result` is (where everything is stored)
`result` is an `IGCResult` object containing **three pandas DataFrames**:

- `result.free_energy` — the “Free Energy” section table
- `result.dispersive_surface_energy` — the “Dispersive Surface Energy” section table
- `result.injection_items` — the “Injection Items” section table
- `result.source_path` — the file path that was parsed

So “where the data lives” = in those 3 DataFrames (in memory while Python is running).

---

## 2) Quick sanity checks (recommended)
### Shapes
```python
result.free_energy.shape
result.dispersive_surface_energy.shape
result.injection_items.shape
```

### Column names
```python
result.free_energy.columns
result.dispersive_surface_energy.columns
result.injection_items.columns
```

### Basic info (dtypes + missingness)
```python
result.free_energy.info()
result.injection_items.info()
```

---

## 3) Viewing data (`head`, `tail`, full table)
### Preview first rows (defaults to 5)
```python
result.free_energy.head()        # first 5 rows
result.free_energy.head(20)      # first 20 rows
```

### Preview last rows (useful to confirm nothing was truncated)
```python
result.free_energy.tail()
result.injection_items.tail()
```

### Show the table (VS Code notebooks will render an interactive table)
```python
result.free_energy
result.injection_items
```

---

## 4) Assign short variable names (recommended)
```python
free = result.free_energy
dse  = result.dispersive_surface_energy
inj  = result.injection_items
```

---

## 5) Filter rows (`query`)
### Filter injections for DECANE
```python
inj_decane = inj.query("Solvent == 'DECANE'")
inj_decane.head(10)
```

### Filter free energy rows for DECANE
Note: column name has a space, so use backticks.
```python
free_decane = free.query("`Solvent Name` == 'DECANE'")
free_decane
```

### Filter by coverage (example for injection_items)
```python
inj.query("`Actual Fractional Surface Coverage` > 0.02")
```

---

## 6) Select columns (keep only what you need)
```python
inj_cols = inj[[
    "ID",
    "Injection Name",
    "Solvent",
    "Actual Fractional Surface Coverage",
    "Peak Area [µV•min]",
    "Column Temperature [Kelvin]",
]]
inj_cols.head()
```

---

## 7) Sort rows
```python
inj_decane.sort_values("Actual Fractional Surface Coverage")
```

---

## 8) Summaries / aggregation
### Count injections per solvent
```python
inj["Solvent"].value_counts()
```

### Average retention volumes by solvent (Free Energy table)
```python
free_summary = (
    free.groupby("Solvent Name")[[
        "Interpolated Retention Volume (Max)",
        "Interpolated Retention Volume (Com)",
    ]]
    .mean()
    .sort_values("Interpolated Retention Volume (Max)", ascending=False)
)
free_summary
```

---

## 9) Save parsed tables to disk
Writes CSVs for each table into a folder you choose.

```python
from pathlib import Path
result.to_csv_dir(Path("parsed_out"))
```

This creates:
- `parsed_out/free_energy.csv`
- `parsed_out/dispersive_surface_energy.csv`
- `parsed_out/injection_items.csv`

---

## 10) Important notebook tip: reload after editing `Parser.py`
If you change `Parser.py` while the notebook kernel is running, reload it:

```python
import importlib, Parser
importlib.reload(Parser)

result = Parser.parse_sugar_se()
```

---

## 11) Example snippet (copy/paste)
```python
from Parser import parse_sugar_se
result = parse_sugar_se()

result.free_energy.head()
result.injection_items.query("Solvent == 'DECANE'").head()
```